In [1]:
#import libraries

import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.offline as pyo
import chart_studio.plotly as py
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
#import datasets

train=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\train.csv')
test=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\test.csv')
meal_info=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\meal_info.csv')
fulfilment_center_info=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\fulfilment_center_info.csv')

In [3]:
#merge datasets
merge1=pd.merge(train,fulfilment_center_info,how='inner',on='center_id')
df=pd.merge(merge1,meal_info,how='inner',on='meal_id')

#reorder columns
df.rename(columns = {'id':'order_id'}, inplace = True)
df=df.loc[:, ['week','region_code','city_code','center_id','center_type','num_orders','checkout_price','cuisine','category','emailer_for_promotion','homepage_featured']]
df=df.sort_values(by=['week','region_code','city_code','center_id','center_type','num_orders'])


In [4]:
#Exploratory Data Analysis

for i in df.columns:
    print(i)
    print(df[i].value_counts())
    print('Unique Values: '+str(len(df.groupby(i).count()))+'\n')

week
122    3359
105    3348
106    3347
140    3332
123    3331
       ... 
13     2838
12     2831
14     2830
7      2795
8      2786
Name: week, Length: 145, dtype: int64
Unique Values: 145

region_code
56    191228
34    116713
77     94612
85     30283
23      6434
71      6293
93      5721
35      5264
Name: region_code, dtype: int64
Unique Values: 8

city_code
590    54746
526    43525
638    20047
522    13459
517    13109
604    13062
699    12098
647    11833
576    11456
614    11332
676    11177
593    10804
685     6993
700     6891
713     6853
579     6843
679     6801
456     6716
703     6706
698     6434
577     6389
599     6296
596     6293
648     6090
602     6057
659     6053
654     6041
628     5983
651     5975
562     5938
615     5892
556     5885
473     5855
561     5846
461     5763
620     5732
609     5721
675     5715
485     5712
658     5591
553     5547
649     5528
680     5379
683     5296
695     5296
632     5292
702     5264
515     5085
478  

In [5]:
#analysing sales over 145 weeks, 77 centers, 51 cities, 8 regions and 3 types of centers.
#the store sells 51 different meals over 14 meal categories.
#the store uses promotional emails and homepage features on the website to drive sales.

In [6]:
#calculate total orders per week
weekly_orders=df.groupby('week',as_index=False)['num_orders'].sum() #group num_orders by week & calculate sum of orders.

#plot the total orders per week
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=weekly_orders.week,y=weekly_orders.num_orders,mode='lines+markers',line=dict(color='firebrick',width=1)))
fig.update_layout(title='Total Number of Orders per Week',xaxis_title='Week',yaxis_title='Number of Orders')



In [7]:
#calculate revenue per week
weekly_revenue=df.groupby('week',as_index=False)['checkout_price'].sum() #group num_orders by week and calculate total revenue

#plot the total orders per week
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=weekly_revenue.week,y=weekly_revenue.checkout_price,mode='lines+markers',line=dict(color='royalblue',width=1)))
fig.update_layout(title='Total Revenue per Week',xaxis_title='Week',yaxis_title='Revenue')

In [8]:
from plotly.subplots import make_subplots

category_num_orders=df.groupby('category',as_index=False)['num_orders'].sum() #group num_orders by week
 
grouped_df=df.groupby('category',as_index=False) #group num_orders by week
category_revenue=grouped_df['checkout_price'].sum() 

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=category_num_orders.category, values=category_num_orders.num_orders, title="Number of orders per Week"),1, 1)
fig.add_trace(go.Pie(labels=category_revenue.category, values=category_revenue.checkout_price, title="Revenue per Week"),1, 2)


In [9]:
#Beverages are the most popular order and bring in the maximum revenue.

In [10]:
#density heatmap shows cuisine vs category vs num_orders
#describes the kind of orders
grouped_df=df.groupby(['cuisine','category'],as_index=False) #group num_orders by week
cuisine_category_orders=grouped_df['num_orders'].sum()
fig=px.density_heatmap(cuisine_category_orders,y='category',x='cuisine',z='num_orders',title='Number of orders in Cuisine x Category',color_continuous_scale='Inferno')
fig.show()

In [11]:
#Four types of beverages are sold, hence the high number of orders.
#All other categories are sold under a single cuisine.

In [12]:
#density heatmap shows category vs week vs num_orders.
#describes the sale of each category over the weeks.
grouped_df=df.groupby(['week','category'],as_index=False) #group num_orders by week
cuisine_category_orders=grouped_df['num_orders'].sum()
fig=px.density_heatmap(cuisine_category_orders,y='category',x='week',z='num_orders',title='Number of orders per week per cuisine',color_continuous_scale='Inferno')
fig.show()

In [19]:
#compare multiple food categories depending on the user's choice
def choose(choices):
    info=['Beverages','Rice Bowl','Sandwich','Pizza','Starters','Desert','Salad','Pasta','Seafood','Biryani','Soup','Fish','Extras','Other Snacks']
    data_frames=[]
    for i in choices:
        data_frames.append(category_orders(info[i]))
    fig=go.Figure()
    for i in range(0,len(data_frames)):
        colors=['salmon','violet','seagreen','yellow','turquoise','limegreen','mistyrose','tomato','lavender','khaki','olive','teal','orange','chocolate']
        fig.add_trace(go.Scatter(x=data_frames[i].week,y=data_frames[i].num_orders,name=info[choices[i]-1],mode='lines+markers',line=dict(color=colors[i],width=1)))
        fig.update_layout(title='Number of Orders per week',xaxis_title='Week',yaxis_title='Number of Orders')
    fig.show()
def category_orders(category):
    grouped_df=df.groupby(['category','week'],as_index=False) #group num_orders by week
    single_category_orders=grouped_df['num_orders'].sum()
    arr=[]
    weeks=0
    for index, row in single_category_orders.iterrows():
        if row['category']==category:
            weeks+=1
            arr.append(row['num_orders'])
    weeks=[i for i in range(1,weeks+1)]
    weekly_category_orders=pd.DataFrame(list(zip(weeks,arr)),columns=['week','num_orders'])
    return weekly_category_orders
    
print("Pick the categories to be compared")
print('1.Beverages \t2.Rice Bowl \t3.Sandwich \t4.Pizza \t5.Starters \t6.Desert \t7.Salad \n8.Pasta \t9.Seafood \t10.Biryani \t11.Soup \t12.Fish \t13.Extras \t14.Other Snacks')
n=int(input("Enter the number of categories"))
choices=[]
print("Enter the categories")
for i in range(n):
    choices.append(int(input()))
choose(choices)

Pick the categories to be compared
1.Beverages 	2.Rice Bowl 	3.Sandwich 	4.Pizza 	5.Starters 	6.Desert 	7.Salad 
8.Pasta 	9.Seafood 	10.Biryani 	11.Soup 	12.Fish 	13.Extras 	14.Other Snacks
Enter the categories


In [14]:
#weekly sales at centers of different types
grouped_df=df.groupby(['center_type','week'],as_index=False) #group num_orders by centre_type and week
center_type_orders=grouped_df['num_orders'].sum() #calculate sum of orders in the grouping
center_type_orders

#plot the total orders per week
a,b,c=[],[],[]
for index, row in center_type_orders.iterrows():
    if row['center_type']=='TYPE_A':
        a.append(row['num_orders'])
    elif row['center_type']=='TYPE_B':
        b.append(row['num_orders'])
    else: #row['center_type']=='TYPE_C':
        c.append(row['num_orders'])
x=list(i for i in range(1,146))
center_type_orders=pd.DataFrame(list(zip(x,a,b,c)),columns=['week','TYPE_A','TYPE_B','TYPE_C'])
fig=go.Figure()
fig.add_trace(go.Scatter(x=center_type_orders.week,y=center_type_orders.TYPE_A,mode='lines+markers',name='TYPE_A',line=dict(color='firebrick',width=1)))
fig.add_trace(go.Scatter(x=center_type_orders.week,y=center_type_orders.TYPE_B,mode='lines+markers',name='TYPE_B',line=dict(color='green',width=1)))
fig.add_trace(go.Scatter(x=center_type_orders.week,y=center_type_orders.TYPE_C,mode='lines+markers',name='TYPE_C',line=dict(color='blue',width=1)))
fig.update_layout(title='Number of Orders per Week in Each Type of Center',xaxis_title='Week',yaxis_title='Revenue')

In [15]:
#Highest number of orders in type_A, followed by type_B and type_C

In [16]:
#effect of promotional emailer and homepage feature on number of orders
grouped_df=df.groupby(['emailer_for_promotion','homepage_featured'],as_index=False) #group num_orders by week
cuisine_category_orders=grouped_df['num_orders'].sum()
cuisine_category_orders


,emailer_for_promotion,homepage_featured,num_orders
0,0,0,82214568
1,0,1,13960753
2,1,0,7684936
3,1,1,15697228


In [17]:
#Promotional emails and homepage features do not increase the number of orders.